In [45]:
import xgboost as xgb
import shap
import pandas as pd
from pathlib import Path
import os
import numpy as np

In [21]:
while "notebooks" in os.getcwd():
    os.chdir("../")

In [23]:
model_xgb = xgb.Booster()
model_xgb.load_model("models/winequality_red_saved.json")

In [24]:
DATA_DIR = Path('data/wine_quality')
file_name = 'winequality_red.csv'
scaled_file_name = 'winequality_red_scaled.csv'
train_file_name = 'train_winequality_red_scaled.csv'
test_file_name = 'test_winequality_red_scaled.csv'

In [28]:
df = pd.read_csv(DATA_DIR / test_file_name, sep=',')
df.info()
#df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         320 non-null    float64
 1   volatile_acidity      320 non-null    float64
 2   citric_acid           320 non-null    float64
 3   residual_sugar        320 non-null    float64
 4   chlorides             320 non-null    float64
 5   free_sulfur_dioxide   320 non-null    float64
 6   total_sulfur_dioxide  320 non-null    float64
 7   density               320 non-null    float64
 8   pH                    320 non-null    float64
 9   sulphates             320 non-null    float64
 10  alcohol               320 non-null    float64
 11  quality               320 non-null    int64  
dtypes: float64(11), int64(1)
memory usage: 30.1 KB


In [29]:

X = df.loc[:, df.columns != 'quality']
y = df.loc[:, df.columns == 'quality']

In [37]:
X = X.iloc[0:2, :]

In [59]:
explainer = shap.TreeExplainer(model_xgb)
shap_values = explainer.shap_values(X)

[16:33:25] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


In [60]:
shap_values

array([[-0.01007007, -0.33913746,  0.00934775,  0.00501736,  0.02439462,
         0.01092996, -0.04359667,  0.00330668, -0.05497914, -0.11723687,
        -0.15097632],
       [-0.01020539, -0.19239832,  0.01088064, -0.05229618, -0.0066524 ,
        -0.05167738, -0.16086785,  0.00572556, -0.0536715 , -0.2724433 ,
        -0.2497845 ]], dtype=float32)

In [62]:
rank

array([[ 1, 10,  9,  8,  6,  0,  7,  3,  2,  5,  4],
       [ 9, 10,  1,  6,  8,  3,  5,  0,  4,  7,  2]])

In [63]:
features

array(['fixed_acidity', 'volatile_acidity', 'citric_acid',
       'residual_sugar', 'chlorides', 'free_sulfur_dioxide',
       'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol'],
      dtype='<U20')

In [61]:
rank = np.argsort(shap_values)
features = np.array(list(X.columns.values))
features[rank]

In [65]:
features[rank].tolist()

[['volatile_acidity',
  'alcohol',
  'sulphates',
  'pH',
  'total_sulfur_dioxide',
  'fixed_acidity',
  'density',
  'residual_sugar',
  'citric_acid',
  'free_sulfur_dioxide',
  'chlorides'],
 ['sulphates',
  'alcohol',
  'volatile_acidity',
  'total_sulfur_dioxide',
  'pH',
  'residual_sugar',
  'free_sulfur_dioxide',
  'fixed_acidity',
  'chlorides',
  'density',
  'citric_acid']]

In [50]:
shap_values.tolist()

[[-0.010070073418319225,
  -0.339137464761734,
  0.00934775359928608,
  0.005017360206693411,
  0.024394623935222626,
  0.010929963551461697,
  -0.04359666630625725,
  0.003306679194793105,
  -0.05497913807630539,
  -0.11723686754703522,
  -0.15097631514072418],
 [-0.010205386206507683,
  -0.1923983246088028,
  0.01088064257055521,
  -0.05229617655277252,
  -0.006652395240962505,
  -0.05167738348245621,
  -0.1608678549528122,
  0.005725556518882513,
  -0.053671497851610184,
  -0.2724432945251465,
  -0.24978449940681458]]

In [68]:
class ShapWrapper:
   
   def _get_ranking_by_name(index_ranking: np.array, names: list[str]) -> list:

        ranking = names[rank].tolist()
        return ranking


   def get_shap_ranking(self, xgb_tree: xgb.Booster, X: pd.DataFrame, return_str = True) -> list:
        explainer = shap.TreeExplainer(xgb_tree)
        shap_values = explainer.shap_values(X)
        ranking = np.argsort(shap_values)
        if return_str:
            ranking = self._get_ranking_by_name(ranking, list(X.columns.values))
        else:
            ranking = ranking.tolist() 
        return ranking

In [67]:
sw = ShapWrapper()
sw.get_shap_ranking()
get_shap_ranking(model_xgb, X)

[16:40:23] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
